CS 543 Fall 2023

Project #1

Date: 10/13/2023

Group Number: 4

**Data Preprocessing**

In [19]:
#imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum, mean, stddev, median, year, month, count, concat_ws, sha2, desc, asc
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, when, lit

In [2]:
#initialize Spark
spark = SparkSession(sc)

In [3]:
#define the path to the CSV file with transactions data
csv_file_path = '/common/users/shared/cs543_group4/HI-Large_Trans.csv'

#load the data as a spark df
data_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

#rename columns
data_df = data_df.withColumnRenamed('Timestamp', 'timestamp') \
                 .withColumnRenamed('From Bank', 'bank_from') \
                 .withColumnRenamed('Account2', 'account_from') \
                 .withColumnRenamed('To Bank', 'bank_to') \
                 .withColumnRenamed('Account4', 'account_to') \
                 .withColumnRenamed('Amount Received', 'amount_to') \
                 .withColumnRenamed('Receiving Currency', 'currency_to') \
                 .withColumnRenamed('Amount Paid', 'amount_from') \
                 .withColumnRenamed('Payment Currency', 'currency_from') \
                 .withColumnRenamed('Payment Format', 'payment_format') \
                 .withColumnRenamed('Is Laundering', 'is_laundering')

#convert timestamp column to date format
data_df = data_df.withColumn("timestamp", to_date(data_df["timestamp"], "yyyy/MM/dd HH:mm"))

#check count of df
raw_data_count = data_df.count()
print('df with %d rows imported' %raw_data_count)

df with 179702229 rows imported


In [4]:
#check for nulls
null_counts = data_df.select([sum(col(c).isNull().cast('int')).alias(c) for c in data_df.columns])
null_counts.show()

23/10/15 08:06:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Timestamp, From Bank, Account, To Bank, Account, Amount Received, Receiving Currency, Amount Paid, Payment Currency, Payment Format, Is Laundering
 Schema: Timestamp, From Bank, Account2, To Bank, Account4, Amount Received, Receiving Currency, Amount Paid, Payment Currency, Payment Format, Is Laundering
Expected: Account2 but found: Account
CSV file: file:///common/users/shared/cs543_group4/HI-Large_Trans.csv


+---------+---------+------------+-------+----------+---------+-----------+-----------+-------------+--------------+-------------+
|timestamp|bank_from|account_from|bank_to|account_to|amount_to|currency_to|amount_from|currency_from|payment_format|is_laundering|
+---------+---------+------------+-------+----------+---------+-----------+-----------+-------------+--------------+-------------+
|        0|        0|           0|      0|         0|        0|          0|          0|            0|             0|            0|
+---------+---------+------------+-------+----------+---------+-----------+-----------+-------------+--------------+-------------+



In [11]:
#check for duplicate records and store them in a df
duplicate_df = data_df.exceptAll(data_df.dropDuplicates())

#count the number of duplicate rows
num_duplicates = duplicate_df.count()
print('Number of duplicate rows: %d' %num_duplicates)

23/10/15 08:16:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Timestamp, From Bank, Account, To Bank, Account, Amount Received, Receiving Currency, Amount Paid, Payment Currency, Payment Format, Is Laundering
 Schema: Timestamp, From Bank, Account2, To Bank, Account4, Amount Received, Receiving Currency, Amount Paid, Payment Currency, Payment Format, Is Laundering
Expected: Account2 but found: Account
CSV file: file:///common/users/shared/cs543_group4/HI-Large_Trans.csv
23/10/15 08:18:06 ERROR Executor: Exception in task 92.0 in stage 45.0 (TID 1644)
java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:349)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(Bu

Py4JJavaError: An error occurred while calling o294.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 90 in stage 45.0 failed 1 times, most recent failure: Lost task 90.0 in stage 45.0 (TID 1642) (jupyter.cs.rutgers.edu executor driver): java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:349)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:127)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:178)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:321)
	at org.apache.spark.util.collec

In [7]:
#check for outliers (+/- 3SD) in amount_to, amount_from columns
amount_from_stats = data_df.select(mean('amount_from').alias('mean_from'), stddev('amount_from').alias('stddev_from'), median('amount_from').alias('median_from')).first()
amount_to_stats = data_df.select(mean('amount_to').alias('mean_to'), stddev('amount_to').alias('stddev_to'), median('amount_to').alias('median_to')).first()

#create outlier dfs that have records with +/- 3SD payment amounts
from_minus_3sd_df = data_df.filter((col('amount_from') < (amount_from_stats.mean_from - 3 * amount_from_stats.stddev_from)))
from_plus_3sd_df = data_df.filter((col('amount_from') > (amount_from_stats.mean_from + 3 * amount_from_stats.stddev_from)))
to_minus_3sd_df = data_df.filter((col('amount_to') < (amount_to_stats.mean_to - 3 * amount_to_stats.stddev_to)))
to_plus_3sd_df = data_df.filter((col('amount_to') > (amount_to_stats.mean_to + 3 * amount_to_stats.stddev_to)))

print('outliers for amount_from are %f +/- %f.  the median for amount_from is %f' %(amount_from_stats.mean_from, amount_from_stats.stddev_from, amount_from_stats.median_from))
print('outliers for amount_to are %f +/- %f.  the median for amount_from is %f' %(amount_to_stats.mean_to, amount_to_stats.stddev_to, amount_to_stats.median_to))

outliers for amount_from are 3974061.499561 +/- 1389274856.969850.  the median for amount_from is 1399.300000
outliers for amount_to are 5606773.502230 +/- 1815450373.623863.  the median for amount_from is 1397.240000


In [ ]:
#convert all amounts to USD so we can compare on the same scale

In [8]:
#create unique id for bank + account number by hashing the bank and account number
data_df = data_df.withColumn("unique_id_from", sha2(concat_ws("_", "bank_from", "account_from"), 256))
data_df = data_df.withColumn("unique_id_to", sha2(concat_ws("_", "bank_to", "account_to"), 256))


In [9]:
#check if receiving currency and payment currency have the same values for encoding

distinct_currency_from = data_df.select('currency_from').distinct().orderBy('currency_from').rdd.flatMap(lambda x: x).collect()
distinct_currency_to = data_df.select('currency_to').distinct().orderBy('currency_to').rdd.flatMap(lambda x: x).collect()

print('Distinct currencies in paid and received are the same: ' + str(distinct_currency_from == distinct_currency_to))

Distinct currencies in paid and received are the same: True


In [20]:
#check number of records by month
data_df = data_df.withColumn('month', month('timestamp'))
data_df = data_df.withColumn('year', year('timestamp'))
count_by_month = data_df.groupBy('month', 'year').agg(count('*').alias('count')).sort(asc('year'), asc('month'))
count_by_month.show()

#documentation shows data is generated for 8/1-11/5 so we can drop all data after 11/5 
#documentation mentions that data after 11/5 are subsequent laundering transactions for chains that started within the timeframe but did not end within the timeframe
aug_nov_df = data_df.filter((col('month') <= 11) & (col('month') >=8))

#get new total count of records
num_rows = aug_nov_df.count()

#get new counts by month and percentage of total data
count_by_month = aug_nov_df.groupBy('month', 'year').agg(count('*').alias('count'), (count('*')/num_rows).alias('percentage')).sort(asc('year'), asc('month'))
count_by_month.show()

+-----+----+--------+
|month|year|   count|
+-----+----+--------+
|    8|2022|59468136|
|    9|2022|56137121|
|   10|2022|54821376|
|   11|2022| 9269215|
|   12|2022|    6132|
|    1|2023|     249|
+-----+----+--------+



+-----+----+--------+-------------------+
|month|year|   count|         percentage|
+-----+----+--------+-------------------+
|    8|2022|59468136|  0.330937729846713|
|    9|2022|56137121| 0.3124007684362301|
|   10|2022|54821376|0.30507870165147055|
|   11|2022| 9269215|0.05158280006558638|
+-----+----+--------+-------------------+



In [ ]:
#convert timestamp to min max


In [ ]:
#encode categorical columns

#scale continuous columns